In [1]:
!pip install pycocotools
!pip install scikit-image

In [2]:
import torch.nn as nn, numpy as np, torchvision.transforms.v2 as T
import torch, dotenv, os

from torch.utils.data import Dataset, DataLoader, Subset
from matplotlib import pyplot as plt
from pycocotools.coco import COCO
from pathlib import Path
from skimage import io

dotenv.load_dotenv()
data_path = Path(os.getenv("DATAPATH"))

python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 8
python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 10
python-dotenv could not parse statement starting at line 11
python-dotenv could not parse statement starting at line 18
python-dotenv could not parse statement starting at line 21
python-dotenv could not parse statement starting at line 28
python-dotenv could not parse statement starting at line 30
python-dotenv could not parse statement starting at line 32
python-dotenv could not parse statement starting at line 33
python-dotenv could not parse statement starting at line 37
python-dotenv could not parse statement starting at line 40
python-dotenv could not parse statement starting at line 42
python-dotenv could not parse statement start

In [3]:
# Hyperparameters
batch_size = 32

In [4]:
class DS(Dataset):
    def __init__(self, annotation_file, categories=['horse']):
        super().__init__()
        self.coco = COCO(data_path/"annotations"/annotation_file)
        self.category_ids = self.coco.getCatIds(categories)
        self.image_ids = self.coco.getImgIds(catIds=self.category_ids)
        self.T = T.Compose([
            T.ToTensor(),
            T.Resize((572, 572))
        ])

    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, index) -> tuple[torch.Tensor, list[torch.Tensor]]:
        img_id = [self.image_ids[index]]
        annotation_ids = self.coco.getAnnIds(imgIds=img_id, catIds=self.category_ids, iscrowd=None)

        return (
            self.T(io.imread(data_path/"val2017"/self.coco.loadImgs(img_id)[0]['file_name'])),
            torch.stack([self.T(self.coco.annToMask(ann)) for ann in self.coco.loadAnns(annotation_ids)], dim=0)
        )

In [5]:
def collate(batch):
    return (
        torch.stack([item[0] for item in batch], dim=0),
        [item[1] for item in batch]
    )

In [6]:
ds_val = DS('instances_val2017.json')
dl_val = DataLoader(ds_val, batch_size, collate_fn=collate)

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!


/home/brambles/anaconda3/lib/python3.13/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [7]:

def crop(tensor: torch.Tensor, size: int):
    offset = (tensor.shape[-1] - size) // 2
    return tensor[:, :, offset:(offset+size), offset:(offset+size)]


class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        s = nn.Sequential

        self.contracting_layers = [
            s(
                nn.Conv2d(3, 64, 3),
                nn.ReLU(),
                nn.Conv2d(64, 64, 3),
                nn.ReLU()
            ),
            s(
                nn.MaxPool2d(2),
                nn.Conv2d(64, 128, 3),
                nn.ReLU(),
                nn.Conv2d(128, 128, 3),
                nn.ReLU(),
            ),
            s(
                nn.MaxPool2d(2),
                nn.Conv2d(128, 256, 3),
                nn.ReLU(),
                nn.Conv2d(256, 256, 3),
                nn.ReLU()
            ),
            s(
                nn.MaxPool2d(2),
                nn.Conv2d(256, 512, 3),
                nn.ReLU(),
                nn.Conv2d(512, 512, 3),
                nn.ReLU()
            )
        ]

        self.center = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(512, 1024, 3),
            nn.Conv2d(1024, 1024, 3),
            nn.ConvTranspose2d(1024, 512, 2, 2)
        )

        self.expanding_layers = [
            s(
                nn.Conv2d(1024, 512, 3),
                nn.ReLU(),
                nn.Conv2d(512, 512, 3),
                nn.ReLU(),
                nn.ConvTranspose2d(512, 256, 2, 2)
            ),
            s(
                nn.Conv2d(512, 256, 3),
                nn.ReLU(),
                nn.Conv2d(256, 256, 3),
                nn.ReLU(),
                nn.ConvTranspose2d(256, 128, 2, 2)
            ),
            s(
                nn.Conv2d(256, 128, 3),
                nn.ReLU(),
                nn.Conv2d(128, 128, 3),
                nn.ReLU(),
                nn.ConvTranspose2d(128, 64, 2, 2)
            ),
            s(
                nn.Conv2d(128, 64, 3),
                nn.ReLU(),
                nn.Conv2d(64, 64, 3),
                nn.ReLU(),
            ),
        ]

        self.final = nn.Conv2d(64, 2, 1)

    def forward(self, x):
        c_layer_outputs = []

        for layer in self.contracting_layers:
            x = layer(x)
            c_layer_outputs.append(x)

        x = self.center(c_layer_outputs[-1])

        for i, layer in enumerate(self.expanding_layers, 1):
            x = layer(torch.concat(
                [crop(c_layer_outputs[-i], x.shape[-1]), x],
                dim=1
            ))
        
        return self.final(x)

In [8]:
model = UNet()

imgs, masks = next(iter(dl_val))
model(imgs).shape

torch.Size([32, 2, 388, 388])

In [9]:
class Base_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        s = nn.Sequential

        self.layers = [
            s(
                nn.Conv2d(3, 64, 3),
                nn.ReLU(),
                nn.Conv2d(64, 64, 3),
                nn.ReLU()
            ),
            s(
                nn.MaxPool2d(2),
                nn.Conv2d(64, 128, 3),
                nn.ReLU(),
                nn.Conv2d(128, 128, 3),
                nn.ReLU(),
            ),
            s(
                nn.MaxPool2d(2),
                nn.Conv2d(128, 256, 3),
                nn.ReLU(),
                nn.Conv2d(256, 256, 3),
                nn.ReLU()
            ),
            s(
                nn.MaxPool2d(2),
                nn.Conv2d(256, 512, 3),
                nn.ReLU(),
                nn.Conv2d(512, 512, 3),
                nn.ReLU()
            )
        ]

        # output layer
        self.output = nn.Conv2d(512, 2, kernel_size=1)
        self.activation = nn.Softmax(dim=1)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.output(x)
        x = self.activation(x)
        return x

base_model = Base_CNN()
base_model(imgs).shape

torch.Size([32, 2, 64, 64])